In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pydicom import dcmread
import os
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import re
from glob import glob
from wad import Dict
import cv2

['../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0000,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0001,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0002,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0003,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0004,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0005,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0006,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0007,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0008,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0009,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0010,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0011,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0012,0000b.dcm',
 '../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_a

In [73]:
DATA_ROOT="../datas/2_Domain_Adaptation_dataset/"
def filt(tgt,str_list):
    return sorted([*filter(lambda x: tgt in x, str_list)])

tgt_dcm=glob(DATA_ROOT+f"testset/**/*.dcm",recursive=True)
tgt_png=glob(DATA_ROOT+f"testset/**/*.dcm",recursive=True)
src_dcm=glob(DATA_ROOT+f"**/*.dcm",recursive=True)
src_dcm=sorted(set(src_dcm)-set(tgt_dcm))
src_png=glob(DATA_ROOT+f"**/*.png",recursive=True)
src_png=sorted(set(src_png)-set(tgt_png))



src_f=Dict(**{
    y_kw:list(zip(filt(x_kw,src_dcm),filt(y_kw,src_png)))
    for x_kw,y_kw in [('CT','CT'),('MRI_Label','MRI_Label')]
})

tgt_f=Dict(**{
    y_kw:list(zip(filt(x_kw,tgt_dcm+src_dcm)))
    for x_kw,y_kw in [('testset','testset'),('MRI_nonLabel','MRI_nonLabel')]
})

In [83]:
def save_trunk(data_label):
    for dcmname,maskname in data_label:
        tmp=maskname.split("/")
        new_dir=join(*tmp[:-2],"NewGT")
        new_fname=join(*tmp[:-2],"NewGT",tmp[-1])
        os.makedirs(new_dir,exist_ok=True)

        dcm = dcmread(dcmname)
        mask = (cv2.imread(maskname)[...,0]==255).astype("uint8")

        image=dcm.pixel_array.astype("float")
        slope = float(dcm.RescaleSlope)
        if slope != 1:
            image = slope * image
        b=image-image.min()
        b=b/(b.max()-b.min())
        ret, thresh = cv2.threshold((b*255).astype("uint8"), 100, 110, 0)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours=np.squeeze(sorted(contours, key=len,reverse=True)[0])
        filled_contour=cv2.fillPoly(np.zeros_like(b,dtype="uint8"), pts =[contours], color=255)

        trunk=np.logical_xor(mask,filled_contour)

        mask=mask*64
        mask[trunk]=128


        cv2.imwrite(new_fname,mask)
        

In [104]:
def filt(tgt,str_list):
    return sorted([*filter(lambda x: tgt in x[0], str_list)])

In [108]:
src_f.CT

[('../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0000,0000b.dcm',
  '../datas/2_Domain_Adaptation_dataset/CT/1/Ground/liver_GT_000.png'),
 ('../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0001,0000b.dcm',
  '../datas/2_Domain_Adaptation_dataset/CT/1/Ground/liver_GT_001.png'),
 ('../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0002,0000b.dcm',
  '../datas/2_Domain_Adaptation_dataset/CT/1/Ground/liver_GT_002.png'),
 ('../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0003,0000b.dcm',
  '../datas/2_Domain_Adaptation_dataset/CT/1/Ground/liver_GT_003.png'),
 ('../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0004,0000b.dcm',
  '../datas/2_Domain_Adaptation_dataset/CT/1/Ground/liver_GT_004.png'),
 ('../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0005,0000b.dcm',
  '../datas/2_Domain_Adaptation_dataset/CT/1/Ground/liver_GT_005.png'),
 ('../datas/2_Domain_Adaptation_dataset/CT/1/DICOM_anon/i0006,0000b.dcm',
  '../datas/2_Domain_Adaptation_dataset/CT/1/Groun

In [105]:
data_label=src_f.CT
for dcmname,maskname in data_label:
    tmp=maskname.split("/")
    new_dir=join(*tmp[:-2],"NewGT")
    new_fname=join(*tmp[:-2],"NewGT",tmp[-1])
    os.makedirs(new_dir,exist_ok=True)

    dcm = dcmread(dcmname)
    mask = (cv2.imread(maskname)[...,0]==255).astype("uint8")
    
    image=dcm.pixel_array.astype("float")
    image[image>5000]=0.
    slope = float(dcm.RescaleSlope)
    if slope != 1:
        image = slope * image
    b=image-image.min()
    b=b/(b.max()-b.min())
    ret, thresh = cv2.threshold((b*255).astype("uint8"), 100, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours=np.squeeze(sorted(contours, key=len,reverse=True)[0])
    filled_contour=cv2.fillPoly(np.zeros_like(b,dtype="uint8"), pts =[contours], color=255)
    if (filled_contour/255).sum()<((b.shape[-1]*b.shape[-2])/3):
        print("trunk wierd:",maskname)
        continue
        
    trunk=np.logical_xor(mask,filled_contour)

    mask=mask*64
    mask[trunk]=128
    

    cv2.imwrite(new_fname,mask)

trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_005.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_012.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_013.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_014.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_015.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_019.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_020.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_021.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_022.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_023.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_082.png
trunk wierd: ../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_103.png
trunk wierd: ../datas/2_Domain_Adaptatio

In [87]:
maskname

'../datas/2_Domain_Adaptation_dataset/CT/10/Ground/liver_GT_000.png'

In [106]:
mask = (cv2.imread(maskname)[...,0]==255).astype("uint8")